## Model Development

Based on the previous work in the other notebooks, we managed to gain some initial insight for what might be contributing features when it comes to determining an online users MBTI personality factors based on what they posted, such as the overall sentiment of their posts, length of their posts, noun and verb frequency of their posts, etc. Here we try to develop a model with more emphasis on performance.

### SMOTE + Word2Vec + Logistic Regression

In the paper *Ryan, G.; Katarina, P.; Suhartono, D. MBTI Personality Prediction Using Machine Learning and SMOTE for Balancing Data Based on Statement Sentences. Information 2023, 14, 217*, a combination of SMOTE oversampling, Word2Vec word embeddings and Logistic Regression was used to achieve an average F1 - Score of 0.8337 across the four dimensions of MBTI personalities, namely:
- Extraversion vs Introversion
- Sensing vs Intuition
- Thinking vs Feeling
- Judgment vs Perception

In this section we will attempt to implement this model from the paper using their methodology.

##### Preprocess Dataset

In [23]:
import pandas as pd
import numpy as np
import os
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize

In [24]:
CSV_DATA_PATH = os.path.join("data", "mbti_2.csv")

def load_csv_data(csv_file_path: str):
    """ Load data from a given csv file into a pandas DataFrame object.

    Args:
    - csv_file_path (str) - The file path of the csv file containing the desired data to load into a pandas DataFrame object.

    Returns:
    - data (pandas.DataFrame) - The loaded data as a pandas DataFrame object.
    """
    assert csv_file_path.endswith(".csv")

    return pd.read_csv(csv_file_path)

def lemmatize_words(text: str):
    """ Lemmatize the input string of text.

    Args:
    - text (str) - The input string of text to lemmatize.

    Returns:
    - lemmatized (str) - The lemmatized version of the string of text that was given to the function.
    """
    lemmatizer = WordNetLemmatizer()

    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized = " ".join(words)

    return lemmatized

def preprocess_df(df: pd.DataFrame):
    """ Helper function for preprocessing the input pandas dataframe by helping clean up input user posts.

    Args:
    - df (pd.DataFrame) - The input dataframe object containing the user posts to clean up.

    Returns:
    - df (pd.DataFrame) - The given dataframe object after cleaning has been applied to the user posts data.
    """   

    # Keep the End Of Sentence characters
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'\.', ' EOSTokenDot ', str(x) + " "))
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'\?', ' EOSTokenQuest ', str(x) + " "))
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'!', ' EOSTokenExs ', str(x) + " "))
    
    # Strip Punctation
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'[\.+]', ".", str(x)))

    # Remove multiple fullstops
    # df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'[^\w\s]','', str(x)))

    # Remove Non-words
    # df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','', str(x)))

    # Convert posts to lowercase
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: str(x).lower())

    # Remove multiple letter repeating words
    # df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','', str(x)))
    
    # Strip trailing whitespaces
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: str(x).strip())

    # Remove rows with no text
    df["posts_no_url"] = df["posts_no_url"].replace('', np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace('nan', np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace("'", np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace("''", np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace('"', np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace('""', np.nan)
    df.dropna(subset=["posts_no_url"], inplace=True)

    # Lemmatize posts
    df["posts_no_url"] = df["posts_no_url"].apply(lemmatize_words)

    # Tokenize posts
    df["posts_no_url_tokens"] = df["posts_no_url"].apply(wordpunct_tokenize)

    return df[["e_vs_i", "s_vs_n", "t_vs_f", "j_vs_p", "type", "posts", "posts_no_url", "posts_no_url_tokens"]]

def binarize_targets(df: pd.DataFrame):
    """ Apply 0/1 labels to the input classes in the df.

    Args:
    - df (pd.DataFrame) - The input dataframe object containing the classes to numerically label.

    Returns:
    - df (pd.DataFrame) - The given dataframe object after numerically labelling classes.
    """
    binary_map = {
        'I': 0, 
        'E': 1, 
        'N': 0, 
        'S': 1, 
        'F': 0, 
        'T': 1, 
        'J': 0, 
        'P': 1
    }

    df["EI"] = df["e_vs_i"].apply(lambda x: binary_map[str(x)])
    df["SN"] = df["s_vs_n"].apply(lambda x: binary_map[str(x)])
    df["TF"] = df["t_vs_f"].apply(lambda x: binary_map[str(x)])
    df["JP"] = df["j_vs_p"].apply(lambda x: binary_map[str(x)])

    df["target_vec"] = df.apply(lambda x: [
        x["EI"],
        x["SN"],
        x["TF"],
        x["JP"]
    ], axis=1)

    return df

def preprocess_dataset(csv_file_path: str = CSV_DATA_PATH):
    """ Load and preprocess data to be used for model development.

    Args:
    - csv_file_path (str) - The file path of the csv file containing the desired data to load into a pandas DataFrame object.

    Returns:
    - numpy arrays of training and test split data.
    """

    # Apply preprocessing to input user posts
    df = load_csv_data(csv_file_path)
    df = preprocess_df(df)

    # Preprocess target labels into a binary vector
    df = binarize_targets(df)

    return df

In [25]:
df = preprocess_dataset()
df.head()

,e_vs_i,s_vs_n,t_vs_f,j_vs_p,type,posts,posts_no_url,posts_no_url_tokens,EI,SN,TF,JP,target_vec
2,I,N,F,J,INFJ,enfp and intj moments https://www.youtube.com...,enfp and intj moment sportscenter not top ten ...,"[enfp, and, intj, moment, sportscenter, not, t...",0,0,0,0,"[0, 0, 0, 0]"
3,I,N,F,J,INFJ,What has been the most life-changing experienc...,what ha been the most life-changing experience...,"[what, ha, been, the, most, life, -, changing,...",0,0,0,0,"[0, 0, 0, 0]"
4,I,N,F,J,INFJ,http://www.youtube.com/watch?v=vXZeYwwRDw8 h...,on repeat for most of today eostokendot,"[on, repeat, for, most, of, today, eostokendot]",0,0,0,0,"[0, 0, 0, 0]"
5,I,N,F,J,INFJ,May the PerC Experience immerse you.,may the perc experience immerse you eostokendot,"[may, the, perc, experience, immerse, you, eos...",0,0,0,0,"[0, 0, 0, 0]"
6,I,N,F,J,INFJ,The last thing my INFJ friend posted on his fa...,the last thing my infj friend posted on his fa...,"[the, last, thing, my, infj, friend, posted, o...",0,0,0,0,"[0, 0, 0, 0]"


The following preprocessing steps were applied to the dataset:
- Converting letters to lowercase
- Removing links
- Removing punctuations
- Removing stopwords

Lematization was also applied after the above steps have been conducted, and the resulting text for each post tokenized.

##### Word Embeddings and Text Vectorization

In [26]:
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

In [27]:
# Train Word2Vec model
w2w_model = Word2Vec(sentences=df["posts_no_url_tokens"], vector_size=100, window=5, min_count=5, epochs=50)

def get_train_test_split(training_fraction: float, df: pd.DataFrame, mbti_dim: str):
    """ Get a training and test dataset split.

    Args:
    - training_proportion (float) - The fraction of the dataset to be used as training data.
    - df (pd.DataFrame) - The pandas dataframe object containing the data to be split into train and test sets.
    - mbti_dim (str) - The column name of the target MBTI personality dimension.

    Returns:
    - Train and test datasets where the target variable is the desired MBTI personality dimension (mbti_dim).
    """

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(df["posts_no_url_tokens"], df[mbti_dim], test_size=(1 - training_fraction), random_state=42)

    X_train = np.array([vectorize(post) for post in X_train])
    X_test = np.array([vectorize(post) for post in X_test])

    return X_train, X_test, y_train, y_test

def vectorize(post: list):
    """ Vectorize a sentence to a vector representation using Word2Vec CBOW.

    Args:
    - post (str) - A post to vectorize. Here, the post is passed into the function as a list of word tokens.

    Returns:
    - Word2Vec word embedding (CBOW).
    """

    # Vectorize sentence
    words_vector = [w2w_model.wv[token] for token in post if token in w2w_model.wv]

    if len(words_vector) == 0:
        return np.zeros(100)
    
    words_vector = np.array(words_vector)
    return words_vector.mean(axis=0)

In [28]:
X_train_EI, X_test_EI, y_train_EI, y_test_EI = get_train_test_split(0.7, df, "EI") 
print(X_train_EI)

[[-0.38337144  0.45117185  0.84649754 ...  0.36549509 -0.44799906
   0.33048698]
 [-0.65333349  0.02470458 -0.22094555 ...  0.27971393 -0.38791531
   0.0902233 ]
 [-0.30711505  0.16958897 -0.77727389 ...  1.09113503 -0.09072065
  -0.20716502]
 ...
 [-0.72033012  0.27641124 -0.2661525  ... -0.90939134  0.32613787
  -0.23198929]
 [ 0.94679403 -1.34951031 -1.78497899 ...  1.55978203  3.53002429
  -0.48310328]
 [-0.01861813  0.86415511  1.20786262 ... -0.46497735 -0.22456919
   0.24400607]]


In [29]:
print(X_train_EI[160])

[-1.04483092  0.96792394  0.09036332 -0.24387008  0.08042677  0.47072533
  0.03963196  0.2330282  -0.57037592  0.14603905  0.8789649  -0.45852929
 -0.67614126 -0.39700082  0.13280533 -0.46391824  0.25712341 -0.32038507
 -0.00278139 -0.3006697  -0.00317743 -1.67118871  1.25529742  0.62555629
 -1.04633796  1.04782712 -0.26789513 -0.17357592  0.9648236  -0.30632395
 -0.17826284 -0.17365976 -0.31818551  1.21937883  0.53019416  0.3174592
  1.21626151  0.49607623 -0.06919578  0.67353165 -0.82898343  0.72627795
 -0.89021772  0.14418924 -1.14582872 -0.30632871 -0.85016423  0.52901191
 -0.30181804  0.39766544 -0.12399467 -0.68355817 -0.26619494 -0.26260328
 -0.96291333  0.14073643  0.68730819  0.33279404  0.37593246  0.95127153
 -0.11948744 -0.30713925  0.11967899 -0.21103275 -0.20507036  0.06893882
 -0.17073165  0.42157441 -0.23363309 -0.00808041 -0.95308846  0.18521222
  0.90139633 -0.18391964 -0.33933204 -0.10248569 -0.3427673   0.7128638
  0.39397308 -0.33363175 -0.74769706  1.05903888  0.0

Here we split out user posts dataset into train and test sets, and vectorized the text data to be used as model inputs as Word2Vec word embeddings. The benefit with using word embeddings is that we can represent words in a numerical format (ML/DL models can only take in numerical input) that allows words with similar meanings to have the same representation, i.e. smilarity of meaning between words is captured in the word embeddings.